<a href="https://colab.research.google.com/github/eduardokern/ML/blob/yolo/notebook/yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Treinamento da rede YOLO

Projeto de criação de uma base de dados e treinamento da rede YOLO .

Seguindo os exemplos de aula, vamos rotular uma base de dados e aplicar o treinamento com a rede YOLO.

Para essa tarefa será necessário utilizar o software LabelMe: http://labelme.csail.mit.edu/Release3.0/ para rotular as imagens.

Também será necessário utilizar a rede YOLO, disponível em: https://pjreddie.com/darknet/yolo/.

Para quem preferir e não quiser rotular uma base de dados, pode usar as imagens já rotuladas do COCO: https://cocodataset.org/#home.

E para quem estiver utilizando um computador que não consiga rodar a rede YOLO, pode utilizar o transfer learning no COLAB: https://colab.research.google.com/drive/1lTGZsfMaGUpBG4inDIQwIJVW476ibXk_#scrollTo=j0t221djS1Gk.

O trabalho deve conter pelo menos duas classes retreinadas para detecção, além das classes já treinadas previamente antes de realizar o transfer learning.  

## Check CUDA release version

In [ ]:
%cd /content
!wget https://developer.download.nvidia.com/compute/cuda/12.4.1/local_installers/cuda_12.4.1_550.54.15_linux.run
!sudo sh cuda_12.4.1_550.54.15_linux.run --silent --toolkit
!rm cuda_12.4.1_550.54.15_linux.run

In [ ]:
# This cell can be commented once you checked the current CUDA version
# CUDA: Let's check that Nvidia CUDA is already pre-installed and which version is it. In some time from now maybe you
!/usr/local/cuda/bin/nvcc --version

## Install cuDNN according to the current CUDA version

In [ ]:
#!sudo apt-get -y install cudnn9-cuda-12

!wget https://developer.download.nvidia.com/compute/cudnn/redist/cudnn/linux-x86_64/cudnn-linux-x86_64-9.7.1.26_cuda12-archive.tar.xz

# We're unzipping the cuDNN files from your Drive folder directly to the VM CUDA folders
!tar -xf cudnn-linux-x86_64-9.7.1.26_cuda12-archive.tar.xz --totals

!cp -vnpr cudnn-linux-x86_64-9.7.1.26_cuda12-archive/include/* /usr/local/cuda/include
!cp -vnpr cudnn-linux-x86_64-9.7.1.26_cuda12-archive/lib /usr/local/cuda

!chmod a+r /usr/local/cuda/include/cudnn.h

# Now we check the version we already installed. Can comment this line on future runs
!cat /usr/local/cuda/include/cudnn.h | grep CUDNN_MAJOR -A 2

!rm cudnn-linux-x86_64-9.7.1.26_cuda12-archive.tar.xz
!rm -rf cudnn-linux-x86_64-9.7.1.26_cuda12-archive

## Installing Darknet

In [ ]:
%cd /content
%rm -rf darknet
!git clone https://github.com/AlexeyAB/darknet/
%cd /content/darknet

#Compile Darknet
#!make CFLAGS+=-w GPU=1 CUDNN=1 OPENCV=1
!make CFLAGS+=-w OPENCV=1

## Some Utils

* imgShow() - Will help us to show an image in the remote VM

In [ ]:
#Show image
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

## Testing if building process worked

In [ ]:
# Execute darknet using YOLOv3 model with pre-trained weights to detect objects on 'person.jpg'
%cd /content/darknet
!./darknet detect cfg/yolov3.cfg /content/drive/MyDrive/Colab/ML/yolo/yolov3.weights data/person.jpg -dont-show

# Show the result using the helper imgShow()
imShow('predictions.jpg')

# Training YOLO on a different dataset

In [ ]:
# Getting a Thermal Dogs and People dataset
!rm -rf /content/dataset
%cd /content
!mkdir dataset
%cd dataset
!wget "https://public.roboflow.com/ds/ooFsXJGdSU?key=DcXUs06019" -O dataset.zip
!unzip -q dataset.zip
!rm dataset.zip

In [ ]:
# Populate train.txt and valid.txt to be used in Yolo training
import glob
import os

def populate_txt_file(root_path, group):
  txt_file = open(f'{root_path}/{group}.txt', 'w')
  for file in glob.iglob(os.path.join(root_path, group, '*.jpg')):
    txt_file.write(f'{os.path.abspath(file)}\n')

dataset_dir = '/content/dataset'
populate_txt_file(dataset_dir, 'train')
populate_txt_file(dataset_dir, 'valid')

In [ ]:
!rm -rf /content/backup
!mkdir /content/backup
%cd /content

#!/content/darknet/darknet detector train /content/drive/MyDrive/Colab/ML/yolo/dataset.data /content/drive/MyDrive/Colab/ML/yolo/yolov3.cfg /content/drive/MyDrive/Colab/ML/yolo/darknet53.conv.74 -dont_show
!/content/darknet/darknet detector train /content/drive/MyDrive/Colab/ML/yolo/dataset.data /content/drive/MyDrive/Colab/ML/yolo/yolov3-tiny.cfg /content/drive/MyDrive/Colab/ML/yolo/darknet53.conv.74 -dont_show

In [ ]:
# Execute darknet using YOLOv3 model with new-trained weights to detect objects on a test image
%cd /content/darknet

import subprocess
import shutil

#yolo_version = 'yolov3'
yolo_version = 'yolov3-tiny'

cfg_path = '/content/drive/MyDrive/Colab/ML/yolo'
#img_test_path = '/content/dataset/test'
#backup_path = '/content/backup'
img_test_path = '/content/drive/MyDrive/Colab/ML/yolo/test'
backup_path = '/content/drive/MyDrive/Colab/ML/yolo'

def test_image(img):
  cmd = [
    './darknet',
    'detector',
    'test',
    f'{cfg_path}/dataset.data',
    f'{cfg_path}/{yolo_version}-detect.cfg',
    f'{backup_path}/{yolo_version}_final.weights',
    f'-dont_show',
    f'-ext_output',
    f'"{img_test_path}/{img}"',
  ]
  print(f'Detecting {img}...')
  result = subprocess.run(cmd, capture_output=True)
  print(result.returncode)
  print(result.stdout.decode())

  predictions_name = f'/content/predictions_{img}'
  shutil.copy('./predictions.jpg', f'"{predictions_name}"')
  imShow(predictions_name)

test_image('IMG_0002 4_jpg.rf.6d1c60bf9f8779f13291c0547ea1ff70.jpg')
test_image('IMG_0027_jpg.rf.888f9a5d05a43267ac39a7c09cd0fc4b.jpg')
test_image('IMG_0106_jpg.rf.a36615f0dcd4336c8340cf7808243766.jpg')
!ls